In [1]:
import re
import json
import csv
import ast

In [2]:
test_string = """THere are fifteen dinosaurs that will live in the back of this truck! 
I work in Downtown Nashville, yo. 
I wish I had more friends; there are lots of people but none of them are like cool. 
Devan wil make us all go. I don't have any more things to say. 
WHAT ARE YOU DOING? 
There's nothing else here for you, but that didn't stop you before. 
The cat will climb that tree. 
The end is nigh! 
George will make us cake?
What are we doing? 
The last one didn't make sense."""

In [3]:
test_string = test_string.replace('\n',' ')

In [4]:
def find_all_overlapping(keyword,string):
    
    string = string.replace('\n',' ').replace('\t',' ')
    
    regex = f"\s*((([^\.!?]*[\.!?]|^)([^\.!?]*[\.!?]|^)[^\.!?]*\s{keyword})\s[^\.!?]*[\.!?]([^\.!?]*[\.!?]|$)([^\.!?]*[\.!?]|$))"

    match = True
    matches = []
    
    while match:
        match = re.search(regex, string)
        if match:
            matches.append(match[0])
            covered_string = match[2]
            string = string.split(covered_string)[1]
    
    return matches
        
find_all_overlapping('will',test_string)

['THere are fifteen dinosaurs that will live in the back of this truck!  I work in Downtown Nashville, yo.  I wish I had more friends; there are lots of people but none of them are like cool.',
 "  WHAT ARE YOU DOING?  There's nothing else here for you, but that didn't stop you before.  The cat will climb that tree.  The end is nigh!  George will make us cake?",
 " climb that tree.  The end is nigh!  George will make us cake? What are we doing?  The last one didn't make sense."]

In [5]:
# Try with the chime dataset.

total_text = ''

with open('./data/chime5_data.csv') as f:
    din = f.read().split('\n')
    
for i,line in enumerate(din):
    if i == 0:
        continue
    try:
        total_text += ' '+line.split(',')[2].replace('"','')
    except:
        print(i)

197411


In [6]:
total_text[:1000]

" We are free. Okay. Let's go cook it. I need a drink. [laughs] Right on. [laughs] It's that survey Yes. Let's get cracked. [inaudible 0:01:21.50] So [noise] It's so much. Oh Um [noise] it's pretty simple. It's just uh dijon mustard What? Is it raspberry vinaigrette? [noise] Or just like [noise] The red wine [noise] Oh Do you do a lot of cooking Re- red wine vinaigrette. Very Chinese. Uh That's good. So Yeah Okay. Oh Who wants to cook bacon? What do you want? What are you drinking? Um You wanna cook Uh not now. I thought you're gonna get you You wanna cut stuff? Yeah. Or I can cook it. I can do it. Whatever you need me Do you wanna fry some He bought the Stella for us Yeah. For sure. All you need is like Or the Maybe four of uh cut cut Uh Do four do do uh day. Do like a I don't know what the difference is. Is it one better than the other? Four slices? [noise] Fry it fry it to be crispy. And then once crispy This one is one of the lagers. But this is Do you want me to save the Yeah s- s

In [7]:
hits_chime = []
hits_chime += find_all_overlapping('will',total_text)
# hits_chime += find_all_overlapping('is going',total_text)
# hits_chime += find_all_overlapping('are going',total_text)
# hits_chime += find_all_overlapping('am going',total_text)

In [8]:
len(hits_chime)

244

In [9]:
import random
random.shuffle(hits_chime)

In [10]:
hits_chime

[" There's your sweets. Oh [laughs] [laughs] uh-huh [laughs] I want more. My body will not appreciate more. Oh. It'll be at sugar overload.",
 " teach how to understand. You will learn how to understand because that's the only way for you to pass. Mhm. Oh Yeah And the thing is is like Mm.",
 "19] Yeah but you found out in your thir- third year of school [laughs] No I got it [laughs] Yeah why don't you That's why was like These are really Oh Because Shandryn Hold on Do you want cheese? Do you have to be a student to get a membership there? No it's like I think it's seventy dollars for the semester or something if you're not a student there Oh even that's so cheap Yeah Yeah That's pretty cheap Babe I think you need to shred more cheese K I think you need to fill yours up Oh I will Oh ok well Yeah Question Do you guys want your peppers Length? Or what Length [inaudible 0:18:50.40] Length and then cut width [laughs] K [laughs] [laughs] Just make it confusing I need to eat And do you want s

In [28]:
with open('chime_hits2.csv','w') as f:
    f.write("line number,directive?,text\n")
    for i,line in enumerate(hits_chime[100:200]):
        f.write(f"{i+1},,{line}\n")

# New Regex that does word counts:

In [11]:
import re


test_string = "THere are fifteen dinosaurs that will live in the back of this truck- will live in the truck, I think. I work in Downtown Nashville, yo. I wish I had more friends; there are lots of people but none of them are like cool. Devan wil make us all go. I don't have any more things to say. WHAT ARE YOU DOING? There's nothing else here for you, but that didn't stop you before. The cat will climb that tree. The end is nigh! George will make us cake? What are we doing? The last one didn't make sense.THere are fifteen dinosaurs that will live in the back of this truck! I work in Downtown Nashville, yo. I wish I had more friends; there are lots of people but none of them are like cool. Devan wil make us all go. I don't have any more things to say. WHAT ARE YOU DOING? There's nothing else here for you, but that didn't stop you before. The greeen cat will climb that tree. The end is nigh! George will make us cake? What are we doing? The last one didn't make sense. Yeah, I suppose I will go to the store."


def find_all_overlapping(keyword,string,overlap=False):
    """
    The way this works is the following: It will take a string, then find the regex pattern and 
    save it. Then, it will remove the last letter of the keyword from that positive hit, so that 
    it won't be found again, and then re-run the search. This will iteratively build the hits list,
    but there will be notifications introduced.
    """
    
    string = string.replace('\n',' ').replace('\t',' ')
    
    regex = "(((\S*\s|^){18}"+keyword+")(\s\S*|$){18})"

    match = True
    matches = []
    
    while match:
        match = re.search(regex, string)
        if match:
            
            # Make more sentence-like...
            actual_match = ''
            first_char = False
            for x in match[0]:
                if x != ' ' and first_char == False:
                    x = x.upper()
                    first_char = True
                actual_match += x
            last_char = False
            for i in range(len(actual_match)-1,-1,-1):
                if actual_match[i] != ' ' and last_char == False:
                    last_char = True
                    if actual_match[i] in ['.',',',';','?','!']:
                        pass
                    else:
                        actual_match = actual_match[:i+1]+'.'
                        
            matches.append(actual_match)
            covered_string = match[2]
            if overlap:
#                 string = string.replace(covered_string,covered_string[:-1])
                string = string.replace(covered_string,covered_string.replace(keyword,'[ALREADY LABELED]'))
            else:
                string = string.split(match[0])[1]
           
    
    return matches
        
print(test_string+"\n\n")

ol = 0
find_all_overlapping('will',test_string,overlap=ol)

THere are fifteen dinosaurs that will live in the back of this truck- will live in the truck, I think. I work in Downtown Nashville, yo. I wish I had more friends; there are lots of people but none of them are like cool. Devan wil make us all go. I don't have any more things to say. WHAT ARE YOU DOING? There's nothing else here for you, but that didn't stop you before. The cat will climb that tree. The end is nigh! George will make us cake? What are we doing? The last one didn't make sense.THere are fifteen dinosaurs that will live in the back of this truck! I work in Downtown Nashville, yo. I wish I had more friends; there are lots of people but none of them are like cool. Devan wil make us all go. I don't have any more things to say. WHAT ARE YOU DOING? There's nothing else here for you, but that didn't stop you before. The greeen cat will climb that tree. The end is nigh! George will make us cake? What are we doing? The last one didn't make sense. Yeah, I suppose I will go to the st

['THere are fifteen dinosaurs that will live in the back of this truck- will live in the truck, I think. I work in Downtown Nashville, yo. I wish I had more friends;',
 "WHAT ARE YOU DOING? There's nothing else here for you, but that didn't stop you before. The cat will climb that tree. The end is nigh! George will make us cake? What are we doing? The last.",
 " One didn't make sense.THere are fifteen dinosaurs that will live in the back of this truck! I work in Downtown Nashville, yo. I wish I had more.",
 "ARE YOU DOING? There's nothing else here for you, but that didn't stop you before. The greeen cat will climb that tree. The end is nigh! George will make us cake? What are we doing? The last.",
 " One didn't make sense. Yeah, I suppose I will go to the store."]

In [23]:
# Use on data.
# result = find_all_overlapping('will',total_text,overlap=False)
# result2 = find_all_overlapping('can you',total_text,overlap=False)
# result3 = find_all_overlapping('thank',total_text,overlap=False)

result = find_all_overlapping('go',total_text,overlap=False)


In [24]:
import random

random.shuffle(result)
print(len(result))
print(result)

721
["His parents' house. But why are you guys ever- w- we're never in that area. What? Don't you go there for his parents' place? Yeah What? Why? [laughs] I just didn't go for You're allowed Oh he.", "Me about the thing I that thing I missed? Oh I don't remember what I told you but go ahead. Oh well Josh was like You know that like toilet building which I found out was like.", "Granville island. He's like [laughs] What? Like read the text. It makes no sense. Um Literally had to go from Sofia to Smyrna This is my turn right here. See Blocked me in the beginning of the.", "Trying to teach I'm like okay I'm gonna play on the wall now. [laughs] Aw. Every time I go now though I'm like she's gonna get mad at me if I'm not holding my st- right? [laughs].", "A there's some sort of messaging that needs to be changed. But B I think she needs to go at like just talking to Tom about it she needs to go after some Two should be okay.", "Didn't hear anything. I was like No [noise] Mm. Yeah. [noise] 

In [25]:
with open('production_data/lever_go.csv','w') as f:
    f.write("line number,directive?,text\n")
    for i,line in enumerate(result):
        f.write(f"{i+1},,{line}\n")

In [5]:
def get_pos_hits(pattern, datafile = 'data/chime5_data.csv'):
    
    with open(datafile) as f:
#         din = f.read().split('\n')
        din = csv.reader(f)

        texts = {}
        poss = {}
        for i,line in enumerate(din):
            if i == 0:
                continue
            texts[i] = line[2]
            poss[i] = line[4]
            
            
    returning_indices = []
    for k,v in poss.items():
        pos = ast.literal_eval(v)
        pos_str = ''.join(pos)
        patt_str = ''.join(pattern)
        returning = False
        if patt_str in pos_str:
            returning = True
            returning_indices.append(k)
    
    
    returning_list = []
    for index in returning_indices:
        try:
            returning_list.append(' '.join([texts[index-2],texts[index-1],texts[index],texts[index+1],texts[index+2]]))
        except KeyError:
            pass
        
    return returning_list

In [6]:
result = get_pos_hits(['PART','VERB'])

In [7]:
import random

random.shuffle(result)
print(len(result))
print(result[:10])

16913
["Like, I'm just like, well. Why can't that be present energies and not based on past or I kind of tend to agree. Or, more it's when I think about it, the idea of me doing life again sounds exhausting to me. It does. Mhm.", "We get lots of light, but not a lot of direct sunlight, so it No, it doesn't get hot at all, really. doesn't get hot, but it also means for growing things you're not gonna get them them burning. Um Yeah. But you might not get big tomatoes, either.", "And it's like uh It's seven weeks at BCIT Long story short, this t- this course is like, just to prep you for the test Oh. okay I don't know, should we oil this? They do it on YouTube so it's air, like", "Yeah. I have I I actually have I didn't Okay, so what we'll do is instead of, like, passing your cards first, just hold your deck first and then we'll do the reveal because if you're doing the chopsticks one you have to put it back, right, and then we'll pass after. Cuz I think it's confusing and then we're not 

In [8]:
with open('production_data/lever_part-verb.csv','w') as f:
    f.write("line number,directive?,text\n")
    for i,line in enumerate(result):
        f.write(f'{i+1},,"{line}"\n')

# FOMC

In [25]:
# Get all of the matches from one of the FMOC docs.
hits_fmoc = []


for i in range(268):
    with open('../fomc_transcripts.jsonlist') as f:
        print(i)
        jsonlist = f.readlines()
        data = json.loads(jsonlist[i])
        speeches = data['speeches']

    for speech in speeches:
        text = speech['text']
        hits_fmoc += find_all_overlapping('will',text)
        hits_fmoc += find_all_overlapping('is going',text)
        hits_fmoc += find_all_overlapping('are going',text)
        hits_fmoc += find_all_overlapping('am going',text)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267


In [27]:
len(hits_fmoc)

33810

In [29]:
import pickle
with open("FOMC_hits.pkl",'wb+') as f:
    pickle.dump(hits_fmoc,f)

In [105]:
# Now, try with an AMI document.

hits_ami = []

with open('../EN2001a.transcript.txt','r') as f:
    lines = f.readlines()
    
for text in lines:
    hits_ami += find_all_overlapping('will',text)
    hits_ami += find_all_overlapping('is going',text)
    hits_ami += find_all_overlapping('are going',text)
    hits_ami += find_all_overlapping('am going',text)

In [114]:
# Now, let's try a book.

hits_hp = []

with open('../HP2.txt','r') as f:
    lines = f.readlines()
    text = ' '.join(lines)

hits_hp += find_all_overlapping('will',text)
hits_hp += find_all_overlapping('is going',text)
hits_hp += find_all_overlapping('are going',text)
hits_hp += find_all_overlapping('am going',text)

hits_fotr = []

with open('../FotR.txt','r') as f:
    lines = f.readlines()
    text = ' '.join(lines)

hits_fotr += find_all_overlapping('will',text)
hits_fotr += find_all_overlapping('is going',text)
hits_fotr += find_all_overlapping('are going',text)
hits_fotr += find_all_overlapping('am going',text)

In [116]:
len(hits_fotr)

502

In [117]:
print(len(hits_hp))
for hit in hits_fotr:
    print(hit,'\n')

8
 But even from the points of view of many who have enjoyed my story there is much that fails to please. It is perhaps not possible in a long tale to please everybody at all points, nor to displease everybody at the same points; for I find from the letters that I have received that the passages or chapters that are to some a blemish are all by others specially approved. The most critical reader of all, myself, now finds many defects, minor and major, but being fortunately under no obligation either to review the book or to write it again, he will pass over these in silence, except one that has been noted by others: the book is too short.       As for any inner meaning or 'message', it has in the intention of the author none. It is neither allegorical nor topical. 

 Smith, belongs rather to the accessory volume.                                   PROLOGUE    This book is largely concerned with Hobbits, and from its pages a reader may discover much of their character and a little of the

In [36]:
output = hits_fmoc
import random
random.shuffle(output)
len(output)

33810

In [37]:
with open('FOMC_hits.txt','w') as f:
    for line in output[:100]:
        f.write(line+'\n')